In [53]:
def update_second_column(input_file, output_file):
    updated_lines = []
    with open(input_file, 'r') as f:
        for line in f:
            if line.strip():  # проверяем, что строка не пустая
                parts = line.split()
                if len(parts) >= 2:
                    second_column_value = parts[2]
                    if '_' in second_column_value:
                        prefix, suffix = second_column_value.rsplit('_', 1)
                        if len(suffix) < 5:
                            suffix = suffix.zfill(5)
                        updated_second_column_value = prefix + '_' + suffix
                        parts[2] = updated_second_column_value
                updated_line = ' '.join(parts) + '\n'
                updated_lines.append(updated_line)

    with open(output_file, 'w') as f:
        f.writelines(updated_lines)

# Пример использования:
input_file = '2024_05_07_full_annotation.tsv'
output_file = 'output_all.txt'
update_second_column(input_file, output_file)

In [1]:
def get_number_from_protein(protein_string):
    # Находим второе появление символа "_" и извлекаем число после него
    second_underscore_index = protein_string.find('_', protein_string.find('_') + 1)
    number_string = protein_string[second_underscore_index + 1:]
    # Преобразуем строку в число, учитывая ведущие нули
    return int(number_string.lstrip('0'))

def write_to_output_file(clust_region_list, output_file):
    with open(output_file, 'a') as f:
        f.write(','.join(clust_region_list) + '\n')

def main(input_file, output_file):
    # Сортируем файл по второй колонке
    with open(input_file, 'r') as f:
        lines = f.readlines()
        # Пропускаем первую строку (хэдер)
        header = lines[0]
        data = sorted(lines[1:], key=lambda x: x.split(' ')[2])

    clust_region = []
    prev_protein_number = None

    for line in data:
        # Разделяем строку по пробелу
        columns = line.strip().split(' ')
        # Проверяем, есть ли четыре столбца
        if len(columns) < 3:
            print("Ошибка: Неверное количество столбцов в строке:", line)
            continue
        protein = columns[2]
        size_clust = int(columns[1])
        clust_region_value = columns[2]
        if size_clust <= 0:
            clust_region_value = 'x' + clust_region_value

        # Получаем число из второй колонки (protein)
        try:
            protein_number = get_number_from_protein(protein)
        except ValueError:
            print("Ошибка: Невозможно извлечь число из строки:", protein)
            continue

        if prev_protein_number is None:
            prev_protein_number = protein_number

        # Проверяем разницу между текущим и предыдущим числами
        if abs(protein_number - prev_protein_number) <= 1:
            # Добавляем значение в лист clust_region
            clust_region.append(clust_region_value + '_' + columns[-2])  # Взять значение из последней колонки
        else:
            # Разница больше 1, записываем текущий список в новый файл
            write_to_output_file(clust_region, output_file)
            # Очищаем список
            clust_region = []
            # Добавляем текущее значение в новый список
            clust_region.append(clust_region_value + '_' + columns[-2])  # Взять значение из последней колонки

        prev_protein_number = protein_number

    # Записываем оставшиеся данные в файл
    write_to_output_file(clust_region, output_file)

if __name__ == '__main__':
    input_file = 'output_all.txt'
    output_file = 'output_file_full_all_ID_PRO.csv'
    main(input_file, output_file)

Обработка последовательностей кластеров

In [2]:
import csv

def reverse_proteins(line):
    proteins = line.strip().split(',')
    inner_index = None
    brex_index = None
    for i, protein in enumerate(proteins):
        if protein.endswith('_inner') or protein.endswith('_brex'):
            inner_index = i
            brex_index = i
            break
    if inner_index is not None:
        downstream_index = None
        for i, protein in enumerate(proteins):
            if protein.endswith('_downstream'):
                downstream_index = i
                break
        if downstream_index is not None and downstream_index < inner_index:
            reversed_proteins = proteins[::-1]
            return ','.join(reversed_proteins)
    return line.strip()

input_file = 'output_file_full_all_ID_PRO.csv'
output_file = 'output_file_full_ordered_all_ID_PRO.csv'

with open(input_file, 'r') as f_in, open(output_file, 'w', newline='') as f_out:
    reader = csv.reader(f_in)
    writer = csv.writer(f_out)
    for line in reader:
        reversed_line = reverse_proteins(','.join(line))
        writer.writerow(reversed_line.split(','))


запись только внутренних

In [3]:
def filter_csv(input_file, output_file):
    filtered_rows = []

    with open(input_file, 'r', newline='') as f_in:
        reader = csv.reader(f_in)
        for row in reader:
            filtered_row = [item for item in row if item.endswith('_brex') or item.endswith('_inner')]
            filtered_rows.append(filtered_row)

    with open(output_file, 'w', newline='') as f_out:
        writer = csv.writer(f_out)
        writer.writerows(filtered_rows)

# Пример использования:
input_file = 'output_file_full_ordered_all_ID_PRO.csv'
output_file = 'output_full_filtered_all_ID_PRO.csv'
filter_csv(input_file, output_file)

In [6]:
def read_txt_file(file_path):
    data_dict = {}
    with open(file_path, 'r') as file:
        # Пропускаем заголовок
        next(file)
        for line in file:
            # Разбиваем строку по пробелам
            parts = line.strip().split()
            # Получаем ключ (первая колонка) и значение (пятая колонка)
            key = parts[2]
            value = parts[3]
            # Добавляем в словарь
            data_dict[key] = value
        data_dict['SINGLETONE'] = 'singletone'
    return data_dict

def process_file(input_file_path, output_file_path, data_dict):
    with open(input_file_path, 'r') as input_file, open(output_file_path, 'w') as output_file:
        for line in input_file:
            keys = line.strip().split(',')
            # Убираем удаление последнего числа в строке
            # count = keys[-1]
            # keys = keys[:-1]
            result_line = []
            for key in keys:
                # Удаляем все после последнего подчеркивания
                key = key.rsplit('_', 1)[0] if '_' in key else key
                if key in data_dict:
                    result_line.append(data_dict[key])
            # Записываем строку в выходной файл
            output_file.write(','.join(result_line) + '\n')

# Пример использования:
file_path = 'output_all.txt'  # Замените на путь к вашему файлу
data_dict = read_txt_file(file_path)

input_file_path = 'output_full_filtered_all_ID_PRO.csv'  # Замените на путь к вашему входному файлу
output_file_path = 'ID_PRO_annotation.csv'  # Замените на путь к вашему выходному файлу
process_file(input_file_path, output_file_path, data_dict)

In [7]:
# Функция для чтения файла и поиска строки
def find_lines(input_file, pattern):
    line_numbers = []
    with open(input_file, 'r') as file:
        for number, line in enumerate(file, 1):
            if pattern in line:
                line_numbers.append(number)
    return line_numbers

# Функция для извлечения строк из другого файла и записи в новый файл
def extract_lines(source_file, line_numbers, output_file):
    with open(source_file, 'r') as file:
        lines = file.readlines()
    with open(output_file, 'w') as file:
        for number in line_numbers:
            # Проверяем, что номер строки не выходит за пределы списка
            if number <= len(lines):
                file.write(lines[number - 1])

# Используем функции
pattern = "BrxA,BrxB,BrxC,PglX,AAA_15,DUF4435,PglZ,BrxL"
input_file = 'ID_PRO_annotation.csv'  # Имя файла, в котором ищем строки
source_file = 'output_full_filtered_all_ID_PRO.csv'  # Имя файла, из которого будем извлекать строки
output_file = 'ID_PRO_AAA15_DUF4435.txt'  # Имя файла, в который запишем результат

# Находим номера строк с заданным шаблоном
line_numbers = find_lines(input_file, pattern)

# Извлекаем строки и записываем в новый файл
extract_lines(source_file, line_numbers, output_file)

In [8]:
# Чтение и обработка первого файла
with open('ID_PRO_AAA15_DUF4435.txt', 'r') as file1, open('coordinates_table_reg_loc.txt', 'r') as file2, open('genomes_AAA15_DUF4435.txt', 'w') as output:
    file2_data = file2.readlines()

    for line in file1:
        elements = line.strip().split(',')
        first_element = elements[0]
        # Обработка первого элемента для получения нужной части
        parts = first_element.split('_')
        if len(parts) > 2:
            processed_element = parts[0] + '_' + parts[1]
            # Поиск во втором файле
            for file2_line in file2_data:
                file2_elements = file2_line.strip().split('\t')
                if len(file2_elements) >= 3 and file2_elements[3] == processed_element:
                    # Запись значения из первой колонки второго файла в новый файл
                    output.write(file2_elements[0] + '\n')
                    break